<a href="https://colab.research.google.com/github/spiketg/Udacity-Facebook-Private-AI-Scholarship-Challenge/blob/master/PATE_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Private Aggregation of Teacher Ensembles (PATE)

![alt text](https://drive.google.com/uc?id=1fZMBYK1Jc6ULanSRQUI0mc4n5HOS5ZTI)

## Importing Libraries

In [0]:
!pip install syft
import torch
from torch.utils.data import Subset

from torchvision import datasets, transforms

import numpy as np

from syft.frameworks.torch.differential_privacy import pate

     |████████████████████████████████| 225kB 8.9MB/s 
     |████████████████████████████████| 389kB 55.1MB/s 
     |████████████████████████████████| 204kB 36.3MB/s 
     |████████████████████████████████| 460kB 59.5MB/s 
     |████████████████████████████████| 2.1MB 52.8MB/s 
     |████████████████████████████████| 81kB 28.2MB/s 
     |████████████████████████████████| 256kB 63.5MB/s 
     |████████████████████████████████| 266kB 55.8MB/s 
     |████████████████████████████████| 51kB 23.1MB/s 
     |████████████████████████████████| 122kB 65.1MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067058 sha256=302fd9f775b7db503342efa49e8afb79c69e9ff6d1d8c0dcb98422cd7e8c3590
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=2ea8848523126988a41d7ac982449d22542ac1ebb74f973157fa3e9b9dbd9c7d
  Stored in di

W0804 22:39:56.938817 139916199278464 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0804 22:39:56.952795 139916199278464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



## Loading the Dataset

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Load the dataset
train_data = datasets.MNIST(root='./data', train=True, 
                                download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False,
                                  download=True, transform=transform)

0it [00:00, ?it/s]

9920512it [00:01, 8145634.30it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129580.66it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 1941966.29it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 49650.15it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Create Data Loaders for each teacher

In [0]:
num_teachers = 100 # we're working with 100 teachers
num_examples = len(train_data) // num_teachers # the size of each teacher's dataset
num_workers = 0 # number of subprocesses for data loading
batch_size = 64 # number of samples per batch

# Split the data among all teachers
teacher_loaders = []

for i in range(num_teachers):
    indices = list(range(i * num_examples, (i+1) * num_examples))
    data = Subset(train_data, indices)
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=num_workers)
    teacher_loaders.append(loader)

## Create Student Training and Test Set

In [0]:
student_train_size = int(len(test_data) * 0.05) # 5% of data for training - 500 samples
student_test_size = int(len(test_data) * 0.05)  # 5% of data for testing - 500 samples

student_train_data = Subset(test_data, list(range(student_train_size)))
student_test_data = Subset(test_data, list(range(student_train_size, student_train_size + student_test_size)))

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size, num_workers=num_workers)
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size, num_workers=num_workers)

## Defining the Neural Network model

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.log_softmax(x, dim=1)
        return x
'''
      
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(model, trainloader, criterion, optimizer, epochs=10):
    
    model.to(device)
    running_loss = 0
    
    for e in range(epochs):
        
        model.train()
        
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            # 1) erase previous gradients (if they exist)
            optimizer.zero_grad()

            # 2) make a prediction
            pred = model.forward(images)

            # 3) calculate how much we missed
            loss = criterion(pred, labels)

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            optimizer.step()

            # 6) log our progress
            running_loss += loss.item()

In [0]:
def predict(model, dataloader):
    outputs = torch.zeros(0, dtype=torch.long).to(device)
    model.to(device)
    model.eval()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    
    return outputs

## Training the Teachers

In [0]:
teacher_models = []

for i in range(num_teachers):
    print("Training Teacher #", i+1)
    model = Net()
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train(model, teacher_loaders[i], criterion, optimizer)
    teacher_models.append(model)

Training Teacher # 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Training Teacher # 2
Training Teacher # 3
Training Teacher # 4
Training Teacher # 5
Training Teacher # 6
Training Teacher # 7
Training Teacher # 8
Training Teacher # 9
Training Teacher # 10
Training Teacher # 11
Training Teacher # 12
Training Teacher # 13
Training Teacher # 14
Training Teacher # 15
Training Teacher # 16
Training Teacher # 17
Training Teacher # 18
Training Teacher # 19
Training Teacher # 20
Training Teacher # 21
Training Teacher # 22
Training Teacher # 23
Training Teacher # 24
Training Teacher # 25
Training Teacher # 26
Training Teacher # 27
Training Teacher # 28
Training Teacher # 29
Training Teacher # 30
Training Teacher # 31
Training Teacher # 32
Training Teacher # 33
Training Teacher # 34
Training Teacher # 35
Training Teacher # 36
Training Teacher # 37
Training Teacher # 38
Training Teacher # 39
Training Teacher # 40
Training Teacher # 41
Training Teacher # 42
Training Teacher # 43
Training Teacher # 44
Training Teacher # 45
Training Teacher # 46
Training Teacher #

## Aggregate Teacher
Make a prediction for each of the teacher's models, add noise, and return the majority vote

In [0]:
# Define epsilon
epsilon = 0.2

In [0]:
preds = torch.torch.zeros((len(teacher_models), student_train_size), dtype=torch.long)

for i, model in enumerate(teacher_models):
    results = predict(model, student_train_loader)
    preds[i] = results

labels = np.array([]).astype(int)
for image_preds in np.transpose(preds):
    label_counts = np.bincount(image_preds, minlength=10)
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    labels = np.append(labels, new_label)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
PATE_labels = labels # Aggregate Teacher Labels   
true_labels = test_data.targets[:student_train_size] # True Labels
PATE_preds = preds # Labels Obtained by Teachers

print(preds.shape)
print(labels.shape)
print("All Teacher Predicted Labels\n", preds)
print("Aggregate Teacher Labels\n", torch.tensor(PATE_labels))
print("True Labels\n", true_labels)

torch.Size([100, 500])
(500,)
All Teacher Predicted Labels
 tensor([[7, 2, 1,  ..., 9, 0, 6],
        [7, 2, 1,  ..., 9, 0, 6],
        [7, 2, 1,  ..., 9, 0, 6],
        ...,
        [7, 2, 1,  ..., 8, 0, 6],
        [7, 3, 1,  ..., 9, 0, 6],
        [7, 6, 1,  ..., 4, 9, 6]])
Aggregate Teacher Labels
 tensor([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 3, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 2, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 8, 1, 7, 3, 7, 8, 7, 9, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 4, 3, 1, 4,
        1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 6, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1,
        7, 1, 8, 2, 0, 4, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 4,
        1, 4, 4, 7, 2, 3, 2, 1, 1, 8, 1, 8, 1, 8, 5, 0, 3, 4, 2, 5, 0, 1, 1, 1,
        0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 

## PATE Analysis

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=PATE_preds, indices=PATE_labels, noise_eps=epsilon, delta=1e-5, moments=10)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

KeyboardInterrupt: ignored

## Training the Student

In [0]:
data = torch.FloatTensor(test_data.data.clone().detach().numpy()[:student_train_size])
targets = torch.tensor(labels)   

test_data.targets[:student_train_size] = targets


student_train_data = Subset(test_data, list(range(student_train_size)))
student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size, num_workers=num_workers)

In [0]:
print(targets)

tensor([7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 3, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 3, 7, 4, 2, 4, 3, 0, 7, 0,
        2, 7, 1, 7, 3, 7, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 4, 3, 1, 4,
        1, 1, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2,
        5, 6, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1,
        7, 1, 8, 2, 0, 4, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 4,
        1, 4, 4, 7, 2, 3, 2, 1, 1, 8, 1, 8, 1, 8, 5, 0, 3, 4, 2, 5, 0, 1, 1, 1,
        0, 9, 0, 8, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0,
        3, 5, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 7, 7, 9, 2, 2, 4, 1,
        5, 4, 8, 9, 2, 5, 0, 6, 4, 2, 9, 1, 9, 5, 7, 7, 2, 1, 2, 0, 8, 1, 7, 7,
        9, 1, 8, 1, 3, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4,
        1, 5, 4, 2, 9, 2, 0, 4, 0, 0, 2,

In [0]:
def train_student(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    
    model.to(device)
    running_loss = 0
    steps = 0
    
    for e in range(epochs):
        
        model.train()
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            steps += 1
            
            # 1) erase previous gradients (if they exist)
            optimizer.zero_grad()

            # 2) make a prediction
            pred = model.forward(images)

            # 3) calculate how much we missed
            loss = criterion(pred, labels)

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            optimizer.step()

            # 6) log our progress
            running_loss += loss.item()
            
            if steps % 25 == 0:
                test_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for images, labels in test_loader:
                        images, labels = images.to(device), labels.to(device)
                        log_ps = model(images)
                        test_loss += criterion(log_ps, labels).item()

                        # Accuracy
                        ps = torch.exp(log_ps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor))
                model.train()
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))
                running_loss = 0
             
            


student_model = Net()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.002)
epochs = 25
train_student(student_model, student_train_loader, student_test_loader, criterion, optimizer, epochs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 4/25..  Training Loss: 6.475..  Test Loss: 1.493..  Test Accuracy: 0.582
Epoch: 7/25..  Training Loss: 3.732..  Test Loss: 0.629..  Test Accuracy: 0.787
Epoch: 10/25..  Training Loss: 1.902..  Test Loss: 0.441..  Test Accuracy: 0.869
Epoch: 13/25..  Training Loss: 1.491..  Test Loss: 0.428..  Test Accuracy: 0.885
Epoch: 16/25..  Training Loss: 1.111..  Test Loss: 0.494..  Test Accuracy: 0.861
Epoch: 19/25..  Training Loss: 1.028..  Test Loss: 0.487..  Test Accuracy: 0.879
Epoch: 22/25..  Training Loss: 0.972..  Test Loss: 0.460..  Test Accuracy: 0.872
Epoch: 25/25..  Training Loss: 0.729..  Test Loss: 0.507..  Test Accuracy: 0.880


## Testing the Student model

In [0]:
test_loss = 0
accuracy = 0
student_model.eval()
with torch.no_grad():
    for images, labels in student_test_loader:
        images, labels = images.to(device), labels.to(device)
        log_ps = student_model(images)
        test_loss += criterion(log_ps, labels).item()

        # Accuracy
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
student_model.train()
print("Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
      "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
running_loss = 0


Test Loss: 0.507..  Test Accuracy: 0.880


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
